# データガバナンス機能に途中保存する

実験終了前の段階で実験をデータガバナンス機能に途中保存します。  
上から順番に実行してください。

## 1. コミットメッセージを変数に入力する

以下のセルを実行して、実験作業のログとして残す短いメッセージを入力してください。  
※入力値に誤りがある場合、もう一度実行することで訂正ができます。

In [ ]:
message = input()

## 2. 実験をデータガバナンス機能に途中保存する

※データの保存先としてAWS S3準拠のオブジェクトストレージを利用する場合は、  
[こちら](util/operate_S3_annex.ipynb)を実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import os
import glob
from util.scripts import utils

# 実験パッケージパスを取得する
%store -r EXPERIMENT_TITLE
experiment_path = '/home/jovyan/experiments/' + EXPERIMENT_TITLE

# Git管理のパスのリストを作成する
%cd $experiment_path
files = os.listdir()
# ディレクトリ一覧からGit-annex管理するディレクトリ(input_dataとoutput_data)を排除する
dirs = [f for f in files if os.path.isdir(f)]
dirs.remove('input_data')
dirs.remove('output_data')
# 実験パッケージ直下のファイルを取得
files = [f for f in files if os.path.isfile(f)]
# Git管理するパスの配列を作成する
files.extend(dirs)
save_path = []
for file in files:
    save_path.append(experiment_path + '/' + file)

# Git-annex管理するパスの配列を作成する
annexed_save_path = [experiment_path + '/input_data', experiment_path + '/output_data']

In [ ]:
import papermill as pm

%cd ~/
# Git-annex管理ファイルを保存
pm.execute_notebook(
    'WORKFLOWS/EX-WORKFLOWS/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = EXPERIMENT_TITLE + '_' + message + ' (1/2)', PATH = annexed_save_path, IS_RECURSIVE = False)
)

In [ ]:
import papermill as pm

%cd ~/
# Git管理ファイルを保存
pm.execute_notebook(
    'WORKFLOWS/EX-WORKFLOWS/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = EXPERIMENT_TITLE + '_' + message + ' (2/2)', TO_GIT = True, PATH = save_path, IS_RECURSIVE = False)
)

## 3. 実験中ワークフロー機能トップページに遷移する

続けてワークフロー機能を実行する場合は、[こちら](../experiment.ipynb)からトップページに遷移できます。  